# Create own dataset from Twitter

AGENDA:
1. Get auth keys for Twitter API
2. Scrape specific account
3. Scrape by keywords
4. Translate from any language to English using GoogleTranslate API
5. Detailed look at the twitter data
6. Store scraped data in csv on GoogleDrive

In [ ]:
!pip install googletrans==4.0.0-rc1

In [2]:
from googletrans import Translator

import pandas as pd
import time
import traceback
import tweepy

# 1. Authentification

First, you need to register yourself as a developer to get an access to Twitter API.

https://developer.twitter.com/

After completing the form you will get consumer/access keys. 

In [14]:
# the values below are fake due to security reason. Substitute yours unique credentials.

# consumer_key, consumer_secret
api_key = "EI9CxYyCdFmRA8F0ajiqhCmqM"
api_secret = "YGU1PX6qHk7Ip8bGq8Wd0HIJzjLBaZ6KyDT4oeLlaZhOMWISJh"
access_token = "1253047286290153472-qcEgVxJI7yaLp80Aj0JS3jAgcwHfeL"
access_secret = "cE8cb2hjDc28aVA7UAWCKeSNisEKVcA9u2YP2ME8zsqwg"

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# 2. Scrape twitter accounts 

The code below let you to scrape tweets from specific tweeter users. 

I have chosen to scrape 20 most recent tweets from three news media accounts. Scraped data to be saved in csv file.

In [ ]:
tweets, news_media = [], ['cnnbrk', 'ArabNewsjp', 'France24_fr']

count = 20

        
try: 
    for news in news_media:
        print(f"Scraping: {news}")
        
        for tweet in api.user_timeline(id = news, count=count):
            status = api.get_status(id = tweet.id, tweet_mode="extended")

            try:
                tweet_text = status.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                tweet_text = status.full_text

            tweets.append((news, tweet.created_at, tweet.id, tweet_text))
            print(tweet_text)
        time.sleep(15)    
    df = pd.DataFrame(tweets, columns = ['news_media', 'created_at', 'tweet_id', 'text'])
    print("Completed.")
    
except BaseException as e:
    traceback.print_exc()

In [31]:
df['tweet_length'] = df["text"].apply(lambda x: len(x))

In [33]:
df.head(7)

,news_media,created_at,tweet_id,text,tweet_length
0,cnnbrk,2022-06-22 16:55:00,1539653191805140992,"John Wood, a top investigator for the January ...",169
1,cnnbrk,2022-06-22 10:26:11,1539555341574328325,Afghanistan earthquake death toll jumps to mor...,124
2,cnnbrk,2022-06-22 06:12:14,1539491432607653888,At least 250 people are feared dead after a ma...,142
3,cnnbrk,2022-06-21 23:12:42,1539385852526268416,Three people were hospitalized with minor inju...,221
4,cnnbrk,2022-06-21 23:07:26,1539384529781198854,A Los Angeles jury has found embattled comedia...,200
5,cnnbrk,2022-06-21 23:03:54,1539383639431061507,"Alabama, Georgia, Virginia and Washington, DC,...",182
6,cnnbrk,2022-06-21 14:38:55,1539256557439000580,The Supreme Court says Maine cannot exclude re...,173


# 3/4. Scrape tweets by query. Translate in English

The code below lets you to scrape tweets by keywords. You get the most recent tweets by the keywords from Twitter stream. Additionally, if the tweet was not in English, I would like to translate them into English. Save all data in csv.

Therefore, I extract for every tweet: 

* time it was created_at
* tweet_id 
* tweet text 
* username (unique tweeter username) 
* name (name written by user)
* location (where the user is from)
* followers_count (number of user's followers)
* original language

In [ ]:
tweets, text_query = [], 'coronavirus OR virus OR covid-19 OR covid19 OR Κορονοϊός'
count = 10

try:
    for tweet in api.search(q=text_query, count=count, result_type='recent',
                           include_entities=True,
                           monitor_rate_limit=True, 
                           wait_on_rate_limit=True):
        # print(f"Raw tweet: {tweet}")
        tweet_text_full, tweet_text_translated = "", ""

        status = api.get_status(id = tweet.id, tweet_mode="extended")
        try:
            tweet_text_full = status.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            tweet_text_full = status.full_text
            
        try:
            tr = Translator()
            
            if tweet.lang and tweet.lang not in ['en', 'und']:


                print(f"Raw tweet: {tweet_text_full}")
                    
                if tweet.text:
                    translated = tr.translate(tweet_text_full)
                    if translated:
                        tweet_text = translated.__dict__()["text"]
            else:
                tweet_text_translated = tweet_text_full
                
        except Exception as e:
            traceback.print_exc()
            pass
        
        tweets.append((tweet.created_at, tweet.id, tweet_text_full, tweet_text_translated, tweet.user.screen_name, tweet.user.name, 
                               tweet.user.location, tweet.user.followers_count, tweet.lang))
        
        df_tr = pd.DataFrame(tweets, columns = ['created_at', 'tweet_id', 'text_raw', 'text_en', 'username', 'name', 'location', 'followers_count', 'lang'])
        time.sleep(3)
        
    print("Completed.")
    
except BaseException as e:
    traceback.print_exc()        

In [ ]:
df.head(10)

In [ ]:
df[['text_raw', 'text_en', 'lang']]

## 5. Explore api.search response

In [ ]:
tweet

## 6. Save data as csv on drive

In [6]:
# save dataset in csv on drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [26]:
path = "/gdrive/My Drive/YouTube/data/"
df.to_csv(path + 'tweets_news_full.csv')
df_tr.to_csv(path + 'tweets_news_full_translated.csv')